# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [21]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [22]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_beneficio_alto_contagio_bajo = pd.read_csv("instancias/beneficio-alto-contagio-bajo/indice.csv");
df_beneficio_bajo_contagio_alto = pd.read_csv("instancias/beneficio-bajo-contagio-alto/indice.csv");
df_bt_mejor_caso = pd.read_csv("instancias/bt-mejor-caso/indice.csv");
df_bt_peor_caso = pd.read_csv("instancias/bt-peor-caso/indice.csv");
df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");
df_bt_vs_d_mejor = pd.read_csv("instancias/bt-vs-d-mejor/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [23]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../distanciamiento_social", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [24]:
experimentos = [];

### Experimento 1
Comaparar Fuerza Bruta con las primeras 30 instancias de bt-peor-caso y bt-mejor-caso.

In [25]:
for n in range(0, 30):
    fila_n = df_bt_peor_caso.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "FB", fila_n["archivo"]]);

for n in range(0, 30):
    fila_n = df_bt_mejor_caso.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "FB", fila_n["archivo"]]);

### Experimento 2
Analizar resultados de Backtracking en las primeras 30 instancias de beneficio-alto-contagio-bajo y beneficio-bajo-contagio-alto.

In [26]:
for n in range(0, df_beneficio_alto_contagio_bajo.shape[0]):
    fila_n = df_beneficio_alto_contagio_bajo.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT", fila_n["archivo"]]);

for n in range(0, df_beneficio_bajo_contagio_alto.shape[0]):
    fila_n = df_beneficio_bajo_contagio_alto.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT", fila_n["archivo"]]);

### Experimento 3
Comparacion de Backtracking con y sin podas para bt-peor-caso y bt-mejor-caso.

In [27]:
for n in range(0, df_bt_mejor_caso.shape[0]):
    fila_n = df_bt_mejor_caso.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT-O", fila_n["archivo"]]);
    
for n in range(0, df_bt_peor_caso.shape[0]):
    fila_n = df_bt_peor_caso.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT-O", fila_n["archivo"]]);

### Experimento 4
Analizar la ejecución de DP para la instancia dinamica.

In [28]:
for n in range(0, df_dinamica.shape[0]):
    fila_n = df_dinamica.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "DP", fila_n["archivo"]]);

### Experimento 5

Comaprar la ejecución de DP vs Backtracking sobre bt-vs-d-mejor y beneficio-alto-contagio-bajo.

In [29]:
for n in range(0, df_bt_vs_d_mejor.shape[0]):
    fila_n = df_bt_vs_d_mejor.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "DP", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "BT", fila_n["archivo"]]);
    
for n in range(0, df_beneficio_alto_contagio_bajo.shape[0]):
    fila_n = df_beneficio_alto_contagio_bajo.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["M"], "DP", fila_n["archivo"]]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [30]:
columnas = ["dataset", "n", "M", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    display(experimento[3] + experimento[4])
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[3], experimento[4]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 8400/8400'

'DPinstancias/beneficio-alto-contagio-bajo/BT-BACB-50.txt'